***PREPARINING THE REQUIREMENTS***
---
Deleting the results having values 'INITIATED' and 'TIMEOUT'
Also deleting rows with having values 'completed' in amount.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('REAL_TD.csv')

df = df[df['TD_SRESULT'] != 'INITIATED']

df = df[df['TD_SRESULT'] != 'TIMEOUT']

df = df[df['TD_DTRAN_AMOUNT'] != 'Completed']

df = df[df['TD_SFRAUD_CB_STATUS'].notna()]

df['TD_DTRAN_AMOUNT'] = df['TD_DTRAN_AMOUNT'].astype('float64')

df['TD_DTRAN_DATE_TIME'] = pd.to_datetime(df['TD_DTRAN_DATE_TIME'])

df.to_csv('REQ.csv', index=False)

In [ ]:
req = pd.read_csv('REQ.csv')

data = req.drop(columns=['TD_SRESULT'])

Creating a seperate dataset without Transaction Result

DATASET EXPLORATION

In [ ]:
print('The columns in the dataset:\n', data.columns)
print('\n')
print('The datatypes in the dataset:\n', data.dtypes)
print('\n')
print('The unique values in the dataset:\n', data.nunique())
print('\n')
print('The shape of the dataset:\n', data.shape)
print('\n')
print('Describing the dataset:\n', data.describe())

Filling missing values

In [ ]:
null_counts = data.isnull().sum()
null_counts

In [ ]:
for column in data.columns:
    # Checking the data type of the column
    column_dtype = data[column].dtype

    # Filling the missing values based on the data type object as rest were filled
    if column_dtype == 'object':
        data[column] = data[column].fillna('Unknown')

# Verifying if all missing values are filled
print(data.isnull().sum())

Making a seperate DataFrame for fraud

In [ ]:
mask = data['TD_SFRAUD_CB_STATUS'] == 1
fraud = data[mask]
fraud.head(10)

,TD_SCARDHOLDER_NAME,TD_SMERCHANT_ID,TD_STERMINAL_ID,TD_SINSTITUTION_ID,TD_SMASKEDCARDNUMBER,TD_SCUST_IP,TD_DTRAN_DATE_TIME,TD_STRAN_CURRENCY,TD_DTRAN_AMOUNT,TD_SFRAUD_CB_STATUS
4969,Angely Cassy,PlaytechM,PlaytechIn,SAEUSA,479432XXXXXX9950,108.60.250.59,2021-12-20 21:04:00,BSD,20.0,1
4993,Angely Cassy,PlaytechM,PlaytechIn,SAEUSA,479432XXXXXX9950,108.60.250.59,2021-12-20 21:09:00,BSD,20.0,1
5256,Robyn Hield,PlaytechM,PlaytechIn,SAEUSA,473702XXXXXX6728,24.244.177.27,2021-12-20 22:18:00,BSD,50.0,1
7151,Benjamin Lightbourne,PlaytechM,PlaytechIn,SAEUSA,523609XXXXXX9680,64.150.255.107,2021-12-21 16:17:00,BSD,200.0,1
8765,Tara Strachan,PlaytechM,PlaytechIn,SAEUSA,479432XXXXXX8269,108.60.253.82,2021-12-22 06:53:00,BSD,100.0,1
10835,Tara Strachan,PlaytechM,PlaytechIn,SAEUSA,479432XXXXXX8269,192.168.182.10,2021-12-23 05:18:00,BSD,100.0,1
11357,Tara Strachan,PlaytechM,PlaytechIn,SAEUSA,479432XXXXXX8269,192.168.182.10,2021-12-23 14:03:00,BSD,100.0,1
12865,Tara Strachan,PlaytechM,PlaytechIn,SAEUSA,479432XXXXXX8269,192.168.182.10,2021-12-24 08:55:00,BSD,100.0,1
14107,Tammy Albury,PlaytechM,PlaytechIn,SAEUSA,468126XXXXXX0012,64.150.199.247,2021-12-25 02:32:00,BSD,100.0,1
16405,Marvin Emmanuel,PlaytechM,PlaytechIn,SAEUSA,479432XXXXXX1913,24.51.126.20,2021-12-27 11:39:00,BSD,100.0,1


Replacing the dataset with new one, which consists of ratios and dummies

In [ ]:
freq = data.drop(columns=['TD_DTRAN_DATE_TIME', 'TD_STRAN_CURRENCY', 'TD_DTRAN_AMOUNT',
                          'TD_SFRAUD_CB_STATUS', 'TD_SINSTITUTION_ID'])
freq

,TD_SCARDHOLDER_NAME,TD_SMERCHANT_ID,TD_STERMINAL_ID,TD_SMASKEDCARDNUMBER,TD_SCUST_IP
0,yugy nbyt,Lawpreneur,Lawpernu01,411111XXXXXX1111,Unknown
1,fd grt,Lawpreneur,Lawpernu01,411111XXXXXX1111,Unknown
2,hfh ghj,Lawpreneur,Lawpernu01,411111XXXXXX1111,Unknown
3,df ghyjgj,Lawpreneur,Lawpernu01,411111XXXXXX1111,Unknown
4,Vinaykumar Bonagiri,Lawpreneur,Lawpernu01,414367XXXXXX5717,Unknown
...,...,...,...,...,...
45070,Partha Barman,PlaytechM,PlaytechIn,411111XXXXXX1111,114.79.161.5
45071,Concerto Test,MER0000406,TER0000406,411111XXXXXX1111,42.111.8.204
45072,Concerto Test,MER0000406,TER0000406,512345XXXXXX0008,114.79.161.5
45073,Unknown,MER0000406,TER0000406,Unknown,192.168.182.10


In [ ]:
table = pd.DataFrame()

table['CARD_HOLDER'] = data['TD_SCARDHOLDER_NAME']
table['MERCHANT_ID'] = data['TD_SMERCHANT_ID']
table['TERMINAL_ID'] = data['TD_STERMINAL_ID']
table['CARD_NUMBER'] = data['TD_SMASKEDCARDNUMBER']
table['CUST_IP'] = data['TD_SCUST_IP']

In [ ]:
for row in range(len(freq)):
  a = data.iloc[row, 0]
  if a in fraud.iloc[:, 0].values:
    a_count_fraud = (fraud.iloc[:, 0] == a).sum()
    a_count_total = (data.iloc[:, 0] == a).sum()
    ratio = (a_count_fraud/a_count_total)
  else:
    ratio = 0
  table.iloc[row, 0] = ratio

In [ ]:
for row in range(len(freq)):
  b = data.iloc[row, 1]
  if b in fraud.iloc[:, 1].values:
    b_count_fraud = (fraud.iloc[:, 1] == b).sum()
    b_count_total = (data.iloc[:, 1] == b).sum()
    ratio = (b_count_fraud/b_count_total)
  else:
    ratio = 0
  table.iloc[row, 1] = ratio

In [ ]:
for row in range(len(freq)):
  c = data.iloc[row, 2]
  if c in fraud.iloc[:, 2].values:
    c_count_fraud = (fraud.iloc[:, 2] == c).sum()
    c_count_total = (data.iloc[:, 2] == c).sum()
    ratio = (c_count_fraud/c_count_total)
  else:
    ratio = 0
  table.iloc[row, 2] = ratio

In [ ]:
for row in range(len(freq)):
  d = data.iloc[row, 3]
  if d in fraud.iloc[:, 3].values:
    d_count_fraud = (fraud.iloc[:, 3] == d).sum()
    d_count_total = (data.iloc[:, 3] == d).sum()
    ratio = (d_count_fraud/d_count_total)
  else:
    ratio = 0
  table.iloc[row, 3] = ratio

In [ ]:
for row in range(len(freq)):
  e = data.iloc[row, 4]
  if e in fraud.iloc[:, 4].values:
    e_count_fraud = (fraud.iloc[:, 4] == e).sum()
    e_count_total = (data.iloc[:, 4] == e).sum()
    ratio = (e_count_fraud/e_count_total)
  else:
    ratio = 0
  table.iloc[row, 4] = ratio

In [ ]:
table

,CARD_HOLDER,MERCHANT_ID,TERMINAL_ID,CARD_NUMBER,CUST_IP
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
45070,0,0.000923,0.000923,0.002252,0
45071,0,0,0,0.002252,0
45072,0,0,0,0.002252,0
45073,0,0,0,0.002252,0


Making DataFrame for dummies

In [ ]:
dummy = pd.DataFrame()

dummy['INSTITUTION_ID'] = data['TD_SINSTITUTION_ID']
dummy['CURRENCY'] = data['TD_STRAN_CURRENCY']

Coverting datetime to month

In [ ]:
data['TD_DTRAN_DATE_TIME'] = pd.to_datetime(data['TD_DTRAN_DATE_TIME'])
dummy['MONTH'] = data['TD_DTRAN_DATE_TIME'].dt.month

label encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
cat_vars=['INSTITUTION_ID', 'CURRENCY', 'MONTH']

label_encoder = LabelEncoder()

for var in cat_vars:
  dummy[var] = label_encoder.fit_transform(dummy[var])

data_vars = dummy.columns.values.tolist()
to_keep = [i for i in data_vars if i in cat_vars]

In [ ]:
cat_vars=['INSTITUTION_ID', 'CURRENCY', 'MONTH']
for var in cat_vars:
 cat_list='var'+'_'+var
 cat_list = pd.get_dummies(dummy[var], prefix=var)
 dummy1 = dummy.join(cat_list)
 dummy = dummy1

data_vars = dummy.columns.values.tolist()
to_keep = [i for i in data_vars if i not in cat_vars]

In [ ]:
final = pd.concat([table['CARD_HOLDER'], table['MERCHANT_ID'], table['TERMINAL_ID'],
                   table['CARD_NUMBER'], table['CUST_IP'], dummy[to_keep],
                   data['TD_DTRAN_AMOUNT'], data['TD_SFRAUD_CB_STATUS']], axis=1)

final.columns.values

array(['CARD_HOLDER', 'MERCHANT_ID', 'TERMINAL_ID', 'CARD_NUMBER',
       'CUST_IP', 'INSTITUTION_ID', 'CURRENCY', 'MONTH',
       'TD_DTRAN_AMOUNT', 'TD_SFRAUD_CB_STATUS'], dtype=object)

In [ ]:
final

,CARD_HOLDER,MERCHANT_ID,TERMINAL_ID,CARD_NUMBER,CUST_IP,INSTITUTION_ID,CURRENCY,MONTH,TD_DTRAN_AMOUNT,TD_SFRAUD_CB_STATUS
0,0,0,0,0,0,0,5,10,1.00,0
1,0,0,0,0,0,0,5,10,1.00,0
2,0,0,0,0,0,0,5,10,1.00,0
3,0,0,0,0,0,0,5,10,1.00,0
4,0,0,0,0,0,0,5,10,1.00,0
...,...,...,...,...,...,...,...,...,...,...
45070,0,0.000923,0.000923,0.002252,0,3,1,8,10.00,0
45071,0,0,0,0.002252,0,3,1,8,10.00,0
45072,0,0,0,0.002252,0,3,8,4,33.06,0
45073,0,0,0,0.002252,0,3,8,4,33.06,0


In [ ]:
print("1\n", final['CARD_HOLDER'].value_counts())
print("2\n", final['MERCHANT_ID'].value_counts())
print("3\n", final['TERMINAL_ID'].value_counts())
print("4\n", final['CARD_NUMBER'].value_counts())
print("5\n", final['CUST_IP'].value_counts())

Deleting values having first five column's value as 0

In [ ]:
final = final[(final['CARD_HOLDER'] != 0) | (final['MERCHANT_ID'] != 0) | (final['TERMINAL_ID'] != 0) | (final['CARD_NUMBER'] != 0) | (final['CUST_IP'] != 0)]
final.shape

(44842, 10)

# ***Eplorative Data Analysis***

In [ ]:
!pip install -U imbalanced-learn

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', size=16)
import seaborn as sns
sns.set(style='white')
sns.set(style='whitegrid', color_codes=True)

from sklearn.model_selection import train_test_split

In [ ]:
data['TD_SFRAUD_CB_STATUS'].value_counts()

In [ ]:
sns.countplot(x='TD_SFRAUD_CB_STATUS', data = data, palette='hls')
plt.show()

In [ ]:
count_valid = len(data[data['TD_SFRAUD_CB_STATUS']==0])
count_fraud = len(data[data['TD_SFRAUD_CB_STATUS']==1])
pct_valid = count_valid/(count_valid + count_fraud)
print('\033[1m percentage of no default is', pct_valid*100)
pct_fraud = count_fraud/(count_valid + count_fraud)
print('\033[1m percentage of default', pct_fraud*100)

In [ ]:
final.groupby('TD_SFRAUD_CB_STATUS').mean()

In [ ]:
graph = pd.crosstab(data.TD_SINSTITUTION_ID, data.TD_SFRAUD_CB_STATUS)
graph.div(graph.sum(1).astype(float), axis=0).loc[graph[1] > 0].plot(kind='bar', stacked=True, figsize=(5, 3))
plt.title('Stacked Bar Chart of Institution_id vs Fraud')
plt.xlabel('Institution_id')
plt.ylabel('Proportion of Transactions')
plt.savefig('Institution_id_vs_def_stack')

In [ ]:
graph = pd.crosstab(data.TD_STRAN_CURRENCY, data.TD_SFRAUD_CB_STATUS)
graph.div(graph.sum(1).astype(float), axis=0).loc[graph[1] > 0].plot(kind='bar', stacked=True, figsize=(10, 5))
plt.title('Stacked Bar Chart of Currency vs Fraud')
plt.xlabel('Currency')
plt.ylabel('Proportion of Transactions')
plt.savefig('Currency_vs_def_stack')

In [ ]:
graph = pd.crosstab(dummy.MONTH, data.TD_SFRAUD_CB_STATUS)
graph.div(graph.sum(1).astype(float), axis=0).loc[graph[1] > 0].plot(kind='bar', stacked=True, figsize=(10, 5))
plt.title('Stacked Bar Chart of Month vs Fraud')
plt.xlabel('Month')
plt.ylabel('Proportion of Transactions')
plt.savefig('Month_vs_def_stack')

In [ ]:
graph = pd.crosstab(data.TD_STERMINAL_ID, data.TD_SFRAUD_CB_STATUS)
graph.div(graph.sum(1).astype(float), axis=0).loc[graph[1] > 0].plot(kind='bar', stacked=True, figsize=(10, 5))
plt.title('Stacked Bar Chart of Terminal_ID vs Fraud')
plt.xlabel('Terminal_ID')
plt.ylabel('Proportion of Transactions')
plt.savefig('Terminal_ID_vs_def_stack')

In [ ]:
graph = pd.crosstab(data.TD_SMERCHANT_ID, data.TD_SFRAUD_CB_STATUS)
graph.div(graph.sum(1).astype(float), axis=0).loc[graph[1] > 0].plot(kind='bar', stacked=True, figsize=(10, 5))
plt.title('Stacked Bar Chart of Merchant_ID vs Fraud')
plt.xlabel('Merchant_ID')
plt.ylabel('Proportion of Transactions')
plt.savefig('Merchant_ID_vs_def_stack')

# ***Model for calculating probability***

Oversampling the data as it is unbalanced

In [ ]:
X = final.drop(['TD_SFRAUD_CB_STATUS'], axis=1)
y = final['TD_SFRAUD_CB_STATUS']

from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

columns = X_train.columns
os_data_X,os_data_y = os.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y = pd.DataFrame(data=os_data_y,columns=['TD_SFRAUD_CB_STATUS'])

#Checking the numbers of our data
print('\033[1m length of oversampled data is ',len(os_data_X))
print('\033[1m Number of valid transaction in oversampled data ',len(os_data_y[os_data_y['TD_SFRAUD_CB_STATUS'] == 0]))
print('\033[1m Number of fraud ',len(os_data_y[os_data_y['TD_SFRAUD_CB_STATUS'] == 1]))
print('\033[1m Proportion of valid data in oversampled data is ',len(os_data_y[os_data_y['TD_SFRAUD_CB_STATUS'] == 0])/len(os_data_X))
print('\033[1m Proportion of default data in oversampled data is ',len(os_data_y[os_data_y['TD_SFRAUD_CB_STATUS'] == 1])/len(os_data_X))

In [ ]:
final_vars = final.columns.values.tolist()
y = ['TD_SFRAUD_CB_STATUS']
X = [i for i in final_vars if i not in y]
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
rfe = RFE(logreg, n_features_to_select = 10)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
os_data_X.columns

In [ ]:
data_X1 = pd.DataFrame({'Feature': os_data_X.columns, 'Importance': rfe.ranking_},)
data_X1.sort_values(by=['Importance'])

In [ ]:
cols=[]
for i in range (0, len(data_X1['Importance'])):
  if data_X1['Importance'][i] == 1:
    cols.append(data_X1['Feature'][i])
print(cols)
print(len(cols))

In [ ]:
os_data_X['CARD_HOLDER'] = os_data_X['CARD_HOLDER'].astype(float)
os_data_X['MERCHANT_ID'] = os_data_X['MERCHANT_ID'].astype(float)
os_data_X['TERMINAL_ID'] = os_data_X['TERMINAL_ID'].astype(float)
os_data_X['CARD_NUMBER'] = os_data_X['CARD_NUMBER'].astype(float)
os_data_X['CUST_IP'] = os_data_X['CUST_IP'].astype(float)

In [ ]:
X_final = os_data_X[cols]
y_final = os_data_y['TD_SFRAUD_CB_STATUS']

In [ ]:
import statsmodels.api as sm
logit_model = sm.Logit(y_final.astype(float), X_final.astype(float))
result = logit_model.fit()
print(result.summary())

In [ ]:
pvalue = pd.DataFrame(result.pvalues, columns = ['p_value'])
pvalue

In [ ]:
pvs = []
for i in range (0, len(pvalue['p_value'])):
  if pvalue['p_value'][i] < 0.05:
    pvs.append(pvalue.index[i])
if 'const' in pvs:
  pvs.remove('const')
else:
  pvs
print(pvs)
print(len(pvs))

In [ ]:
X = os_data_X[pvs]
y = os_data_y['TD_SFRAUD_CB_STATUS']
logit_model = sm.Logit(y_final.astype(float), X_final.astype(float))
result = logit_model.fit()
print(result.summary())

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X_final.astype(float), y_final.astype(float), test_size=0.30, random_state=42)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(accuracy_score(y_test, y_pred)))
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
print(classification_report(y_test, y_pred))

In [ ]:
demo = pd.DataFrame()
X_final['EST'] = logreg.predict_proba(X_final[X_train.columns])[:,1]
X_final[['CARD_HOLDER', 'EST']].head(10)

In [ ]:
X_train.columns

In [ ]:
X_final


In [ ]:
new_data = np.array([0.104790, 0.000000, 0.000000, 0.000000, 0.000000, 3, 1, 11, 100.000000]).reshape(1, -1)
new_pred=logreg.predict_proba(new_data)[:,1][0]
print("\033[1m New transaction has a {:.2%}".format(new_pred), "chance of charge back")

***UPDATED DATAFRAME***

In [ ]:
from sklearn.model_selection import train_test_split

X1 = final.drop(['TD_SFRAUD_CB_STATUS', 'MERCHANT_ID', 'INSTITUTION_ID', 'CURRENCY', 'MONTH', 'TD_DTRAN_AMOUNT'], axis=1)
y1 = final['TD_SFRAUD_CB_STATUS']

from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=42)

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, random_state=42)

columns = X1_train.columns
os_data_X1, os_data_y1 = os.fit_resample(X1_train, y1_train)
os_data_X1 = pd.DataFrame(data = os_data_X1, columns = columns )
os_data_y1 = pd.DataFrame(data = os_data_y1, columns = ['TD_SFRAUD_CB_STATUS'])

#Checking the numbers of our data
print('\033[1m length of oversampled data is ',len(os_data_X1))
print('\033[1m Number of valid transaction in oversampled data ',len(os_data_y1[os_data_y1['TD_SFRAUD_CB_STATUS'] == 0]))
print('\033[1m Number of fraud ',len(os_data_y1[os_data_y1['TD_SFRAUD_CB_STATUS'] == 1]))
print('\033[1m Proportion of valid data in oversampled data is ',len(os_data_y1[os_data_y1['TD_SFRAUD_CB_STATUS'] == 0])/len(os_data_X1))
print('\033[1m Proportion of default data in oversampled data is ',len(os_data_y1[os_data_y1['TD_SFRAUD_CB_STATUS'] == 1])/len(os_data_X1))

 length of oversampled data is  62630
 Number of valid transaction in oversampled data  31315
 Number of fraud  31315
 Proportion of valid data in oversampled data is  0.5
 Proportion of default data in oversampled data is  0.5


In [ ]:
final_vars1 = final.columns.values.tolist()
y1 = ['TD_SFRAUD_CB_STATUS']
X1 = [i for i in final_vars1 if i not in y1]
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
rfe1 = RFE(logreg, n_features_to_select = 4)
rfe1 = rfe1.fit(os_data_X1, os_data_y1.values.ravel())
print(rfe1.support_)
print(rfe1.ranking_)

[ True  True  True  True]
[1 1 1 1]


In [ ]:
data_X11 = pd.DataFrame({'Feature': os_data_X1.columns, 'Importance': rfe1.ranking_},)
data_X11.sort_values(by=['Importance'])

,Feature,Importance
0,CARD_HOLDER,1
1,TERMINAL_ID,1
2,CARD_NUMBER,1
3,CUST_IP,1


In [ ]:
cols1=[]
for i in range (0, len(data_X11['Importance'])):
  if data_X11['Importance'][i] == 1:
    cols1.append(data_X11['Feature'][i])
print(cols1)
print(len(cols1))

['CARD_HOLDER', 'TERMINAL_ID', 'CARD_NUMBER', 'CUST_IP']
4


In [ ]:
os_data_X1['CARD_HOLDER'] = os_data_X1['CARD_HOLDER'].astype(float)
os_data_X1['TERMINAL_ID'] = os_data_X1['TERMINAL_ID'].astype(float)
os_data_X1['CARD_NUMBER'] = os_data_X1['CARD_NUMBER'].astype(float)
os_data_X1['CUST_IP'] = os_data_X1['CUST_IP'].astype(float)

In [ ]:
X1_final = os_data_X1[cols1]
y1_final = os_data_y1['TD_SFRAUD_CB_STATUS']

In [ ]:
import statsmodels.api as sm
logit_model = sm.Logit(y1_final, X1_final)
result1 = logit_model.fit()
print(result1.summary())

Optimization terminated successfully.
         Current function value: 0.038407
         Iterations 14
                            Logit Regression Results                           
Dep. Variable:     TD_SFRAUD_CB_STATUS   No. Observations:                62630
Model:                           Logit   Df Residuals:                    62626
Method:                            MLE   Df Model:                            3
Date:                 Tue, 20 Jun 2023   Pseudo R-squ.:                  0.9446
Time:                         05:05:36   Log-Likelihood:                -2405.4
converged:                        True   LL-Null:                       -43412.
Covariance Type:             nonrobust   LLR p-value:                     0.000
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
CARD_HOLDER    58.8359      1.354     43.445      0.000      56.182      61.490
TERMINAL_ID  -310

In [ ]:
pvalue1 = pd.DataFrame(result1.pvalues, columns = ['p_value1'])
pvalue1

,p_value1
CARD_HOLDER,0.000000e+00
TERMINAL_ID,2.058370e-37
CARD_NUMBER,0.000000e+00
CUST_IP,5.425654e-172


In [ ]:
pvs1 = []
for i in range (0, len(pvalue1['p_value1'])):
  if pvalue1['p_value1'][i] < 0.05:
    pvs1.append(pvalue1.index[i])
if 'const1' in pvs1:
  pvs1.remove('const1')
else:
  pvs1
print(pvs1)
print(len(pvs1))

['CARD_HOLDER', 'TERMINAL_ID', 'CARD_NUMBER', 'CUST_IP']
4


In [ ]:
X1 = os_data_X1[pvs1]
y1 = os_data_y1['TD_SFRAUD_CB_STATUS']
logit_model = sm.Logit(y1_final, X1_final)
result1 = logit_model.fit()
print(result1.summary())

Optimization terminated successfully.
         Current function value: 0.038407
         Iterations 14
                            Logit Regression Results                           
Dep. Variable:     TD_SFRAUD_CB_STATUS   No. Observations:                62630
Model:                           Logit   Df Residuals:                    62626
Method:                            MLE   Df Model:                            3
Date:                 Tue, 20 Jun 2023   Pseudo R-squ.:                  0.9446
Time:                         05:05:37   Log-Likelihood:                -2405.4
converged:                        True   LL-Null:                       -43412.
Covariance Type:             nonrobust   LLR p-value:                     0.000
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
CARD_HOLDER    58.8359      1.354     43.445      0.000      56.182      61.490
TERMINAL_ID  -310

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X1_train, X1_test, y1_train, y1_test = train_test_split(X1_final, y1_final, test_size=0.30, random_state=42)
logreg = LogisticRegression()
logreg.fit(X1_train, y1_train)

LogisticRegression()

In [ ]:
X1_test

,CARD_HOLDER,TERMINAL_ID,CARD_NUMBER,CUST_IP
31120,0.000000,0.000923,0.002252,0.000000
49054,0.600000,0.008211,0.002252,0.391304
55910,0.104790,0.008211,0.002252,0.250000
33802,1.000000,0.008211,0.002252,0.900000
9683,0.000000,0.000923,0.002252,0.000000
...,...,...,...,...
34204,0.199963,0.006963,0.002252,1.000000
48725,0.800000,0.000923,0.002252,1.000000
22248,0.000000,0.008211,0.002252,0.000000
4915,0.000000,0.000923,0.002252,0.000000


In [ ]:
y_pred = logreg.predict(X1_test)

In [ ]:
weights = np.array([1.0, 1.0, -50.0, 1.0])

weighted_features = X1_test * weights


In [ ]:
weighted_features

,CARD_HOLDER,TERMINAL_ID,CARD_NUMBER,CUST_IP
31120,0.000000,0.000923,-0.112618,0.000000
49054,0.600000,0.008211,-0.112618,0.391304
55910,0.104790,0.008211,-0.112618,0.250000
33802,1.000000,0.008211,-0.112618,0.900000
9683,0.000000,0.000923,-0.112618,0.000000
...,...,...,...,...
34204,0.199963,0.006963,-0.112618,1.000000
48725,0.800000,0.000923,-0.112618,1.000000
22248,0.000000,0.008211,-0.112618,0.000000
4915,0.000000,0.000923,-0.112618,0.000000


In [ ]:
y_pred1 = logreg.predict(weighted_features)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(accuracy_score(y1_test, y_pred1)))
confusion_matrix1 = confusion_matrix(y1_test, y_pred1)
print(confusion_matrix1)
print(classification_report(y1_test, y_pred1))

Accuracy of logistic regression classifier on test set: 0.99
[[9293   33]
 [ 174 9289]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      9326
           1       1.00      0.98      0.99      9463

    accuracy                           0.99     18789
   macro avg       0.99      0.99      0.99     18789
weighted avg       0.99      0.99      0.99     18789



In [ ]:
X1_final = X1_final.drop('EST1', axis=1)
X1_final = X1_final * weights


In [ ]:
X1_final['EST1'] = logreg.predict_proba(X1_final[X1_train.columns])[:,1]
X1_final[['CARD_HOLDER', 'EST1']].head(10)

,CARD_HOLDER,EST1
0,0.0,0.042611
1,0.0,0.042539
2,0.0,0.042611
3,0.0,0.042539
4,0.0,0.042611
5,0.0,0.042616
6,0.0,0.042611
7,0.0,0.042611
8,0.0,0.042539
9,0.0,0.042616


In [ ]:
X1_final

,CARD_HOLDER,TERMINAL_ID,CARD_NUMBER,CUST_IP,EST1
0,0.00000,0.000923,-5.630887,0.000000,0.042611
1,0.00000,0.008211,-5.630887,0.000000,0.042539
2,0.00000,0.000923,-5.630887,0.000000,0.042611
3,0.00000,0.008211,-5.630887,0.000000,0.042539
4,0.00000,0.000923,-5.630887,0.000000,0.042611
...,...,...,...,...,...
62625,0.10479,0.008211,-5.630887,0.518519,0.999909
62626,0.10479,0.008211,-5.630887,0.462050,0.999750
62627,1.00000,0.012667,-20.963275,1.000000,1.000000
62628,0.10479,0.008211,-5.630887,0.518519,0.999909


In [ ]:
weights = np.array([1.0, 1.0, -50.0, 1.0])

new_data1 = np.array([0.10000, 0.008211, 0.444444, 0.000000]).reshape(1, -1)
new_data1 = new_data1 * weights
print(new_data1)
new_pred1 = logreg.predict_proba(new_data1)[:,1][0]

print("\033[1m New transaction has a {:.2%}".format(new_pred1), "chance of charge back")

[[ 1.00000e-01  8.21100e-03 -2.22222e+01  0.00000e+00]]
 New transaction has a 90.36% chance of charge back


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
